In [64]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT

from op_fourier_trafo import *
from tools.classical import *
from tools.quantum import *
from jump_ops import sigmam_LCU

In [65]:
num_qubits = 3
eps = 0
bohr_bound = 0

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=False)

Original spectrum:  [-4.     -3.4641 -2.8284 -2.      2.      2.8284  3.4641  4.    ]
Ideal spectrum:  [-0.      0.067   0.1464  0.25    0.75    0.8536  0.933   1.    ]
Nonrescaled coefficients:  [1. 1. 1. 1.]
Rescaled coefficients:  [0.125 0.125 0.125 0.125]


In [70]:
qr_sys = QuantumRegister(num_qubits, 'sys')
qr_b = QuantumRegister(1, 'b')
cr_b = ClassicalRegister(1, 'cb')
circ = QuantumCircuit(qr_sys, qr_b, cr_b)

jump_op = sigmam_LCU()
print(Operator(jump_op))
np.random.seed(666)
randstate_better = np.zeros(2**(num_qubits))
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
randstate_better /= np.linalg.norm(randstate_better)
circ.initialize(randstate_better, qr_sys)
circ.barrier()

circ.compose(jump_op, [qr_sys[1], qr_b[0]], inplace=True)

circ_sv = Statevector(circ)

Operator([[0.00000000e+00+0.00000000e+00j, 1.11022302e-16+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j, 7.07106781e-01+7.07106781e-01j],
          [1.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j],
          [0.00000000e+00+0.00000000e+00j, 7.07106781e-01+7.07106781e-01j,
           0.00000000e+00+0.00000000e+00j, 0.00000000e+00-1.11022302e-16j],
          [0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+1.00000000e+00j, 0.00000000e+00+0.00000000e+00j]],
         input_dims=(2, 2), output_dims=(2, 2))


In [67]:
zero_projector = pad_term([qt.Qobj(np.array([[1, 0], [0, 0]]))], num_qubits + 1, num_qubits).full()
projected_sv = Statevector(zero_projector @ circ_sv.data)
projected_sys_sv = partial_trace(projected_sv, [num_qubits])

In [68]:
sigmam = np.array([[0, 0], [1, 0]])
padded_sigmam = pad_term([qt.Qobj(sigmam)], num_qubits, 1).full()
state_we_want = padded_sigmam @ randstate_better
state_we_want /= np.linalg.norm(state_we_want)
state_we_want = DensityMatrix(state_we_want)

dist_to_what_we_want = np.linalg.norm(projected_sys_sv - state_we_want)
print(dist_to_what_we_want)
# state_fidelity(projected_sys_sv, state_we_want, validate=False)

0.75
